# Test Processing of a single cycling data

The cycling data will include both RPTs and information during cycling. First, split up these two pieces. Then, for each piece, parse the data.

Compile information on UMBLFEB2022 formation cells

1/17/2023

In [2]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib ipympl

from scipy import signal, ndimage
import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

In [3]:
plotter.initialize(plt)

In [4]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


In [80]:
device_name = 'UMBL2022FEB_CELL152028'

test_list, aux_list = vh.get_test_names('UMBL2022FEB_CELL152028')

cyc_test = []
for test_name in test_list:
    if 'CYC' in test_name:
        cyc_test.append(test_name)
        
assert not len(cyc_test) == 0, 'No cycling test found.'
assert not len(cyc_test) > 1, 'More than one cycling test found.'

cyc_test = cyc_test[0]
cyc_test

'UMBL2022FEB_CELL152028_CYC_1C1CR1_P45C_5P0PSI_20220804_R1'

In [6]:
df = vh.get_cycler_data(cyc_test)
df

,charge_capacity_ah,charge_energy_wh,voltage_v,discharge_capacity_ah,discharge_energy_wh,step_time_s,step_index,test_time_s,current_a,cycle_index,datetime
0,0.000000,0.000000,3.431613,0.000000,0.000000,5.404330e-02,1,5.404310e-02,0.0,1,2022-08-04 10:33:31-04:00
1,0.000000,0.000000,3.431613,0.000000,0.000000,1.006521e+01,1,1.006521e+01,0.0,1,2022-08-04 10:33:41-04:00
2,0.000000,0.000000,3.431613,0.000000,0.000000,2.007375e+01,1,2.007375e+01,0.0,1,2022-08-04 10:33:51-04:00
3,0.000000,0.000000,3.431776,0.000000,0.000000,3.008491e+01,1,3.008491e+01,0.0,1,2022-08-04 10:34:01-04:00
4,0.000000,0.000000,3.431613,0.000000,0.000000,4.009819e+01,1,4.009819e+01,0.0,1,2022-08-04 10:34:11-04:00
...,...,...,...,...,...,...,...,...,...,...,...
1668593,1.250625,5.238502,3.393764,1.821491,5.876043,1.005120e+06,31,1.514279e+07,0.0,1578,2023-02-01 11:09:01-05:00
1668594,1.250625,5.238502,3.393927,1.821491,5.876043,1.005130e+06,31,1.514280e+07,0.0,1578,2023-02-01 11:09:11-05:00
1668595,1.250625,5.238502,3.393927,1.821491,5.876043,1.005140e+06,31,1.514281e+07,0.0,1578,2023-02-01 11:09:21-05:00
1668596,1.250625,5.238502,3.393927,1.821491,5.876043,1.005150e+06,31,1.514282e+07,0.0,1578,2023-02-01 11:09:31-05:00


In [7]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(8, nrows * 3), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(df[xvar], df[yvar])
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
RPT_STEP_INDEX_START         = 12
RPT_STEP_INDEX_END           = 31
RPT_STEP_INDEX_C20_CHARGE    = 26
RPT_STEP_INDEX_C20_DISCHARGE = 23
RPT_STEP_INDEX_VOLTAGE_RELAX_TOP_12HR = 27 # voltage decay after CV hold at 4.2V and 12 hours
RPT_STEP_INDEX_VOLTAGE_RELAX_BOT_2P5HR = 13 # voltage rebound after CC discharge to 3.0V and 2.5 hours


df_cyc = df[df['step_index'] < RPT_STEP_INDEX_START]
df_rpt = df[(df['step_index'] >= RPT_STEP_INDEX_START) & (df['step_index'] <= RPT_STEP_INDEX_END)]

In [105]:
# Parsing the cycle by cycle metrics

df_agg = df.groupby('cycle_index').agg('max')

# Cumulative metrics
df_by_cyc = pd.DataFrame()
df_by_cyc.index = df_agg.index

df_by_cyc['charge_capacity_ah'] = df_agg['charge_capacity_ah']
df_by_cyc['charge_energy_wh'] = df_agg['charge_energy_wh']
df_by_cyc['discharge_capacity_ah'] = df_agg['discharge_capacity_ah']
df_by_cyc['discharge_energy_wh'] = df_agg['discharge_energy_wh']
df_by_cyc['tot_charge_capacity_ah'] = df_agg['charge_capacity_ah'].cumsum()
df_by_cyc['tot_charge_energy_wh'] = df_agg['charge_energy_wh'].cumsum()
df_by_cyc['tot_discharge_capacity_ah'] = df_agg['discharge_capacity_ah'].cumsum()
df_by_cyc['tot_discharge_energy_wh'] = df_agg['discharge_energy_wh'].cumsum()

# Hide the data during RPTs
df_by_cyc['charge_capacity_ah'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['charge_energy_wh'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['discharge_capacity_ah'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['discharge_energy_wh'].loc[df_rpt['cycle_index'].unique()] = np.NaN


# Parse metrics by step type
CYC_STEP_INDEX_CHARGE_CC = 3
CYC_STEP_INDEX_CHARGE_CV = 4
CYC_STEP_INDEX_CHARGE_REST = 5
CYC_STEP_INDEX_DISCHARGE_CC = 6
CYC_STEP_INDEX_DISCHARGE_REST = 7

charge_cc_time_s = []
charge_cv_time_s = []
charge_rest_delta_voltage_v = []
discharge_cc_time_s = []
discharge_rest_delta_voltage_v = []

for row in df_by_cyc.iterrows():

    this_df = df[df['cycle_index'] == row[0]]
    
    # Charge CC
    df_chgcc = this_df[this_df['step_index'] == CYC_STEP_INDEX_CHARGE_CC]
    
    if df_chgcc.empty:
        charge_cc_time_s.append(np.NaN)
    else:
        charge_cc_time_s.append(df_chgcc['test_time_s'].tail(1).item() - \
                                df_chgcc['test_time_s'].head(1).item())

    # Charge CV
    df_chgcv = this_df[this_df['step_index'] == CYC_STEP_INDEX_CHARGE_CV]
    
    if df_chgcv.empty:
        charge_cv_time_s.append(np.NaN)
    else:
        charge_cv_time_s.append(df_chgcv['test_time_s'].tail(1).item() - \
                                df_chgcv['test_time_s'].head(1).item())
    
    # Charge Rest
    df_chgrest = this_df[this_df['step_index'] == CYC_STEP_INDEX_CHARGE_REST]
    
    if df_chgrest.empty:
        charge_rest_delta_voltage_v.append(np.NaN)
    else:
        charge_rest_delta_voltage_v.append(df_chgrest['voltage_v'].tail(1).item() - \
                                           df_chgrest['voltage_v'].head(1).item())
                
    # Discharge CC
    df_dchcc = this_df[this_df['step_index'] == CYC_STEP_INDEX_DISCHARGE_CC]
    
    if df_dchcc.empty:
        discharge_cc_time_s.append(np.NaN)
    else:
        discharge_cc_time_s.append(df_dchcc['test_time_s'].tail(1).item() - \
                                   df_dchcc['test_time_s'].head(1).item())

    
    # Discharge Rest
    df_dchrest = this_df[this_df['step_index'] == CYC_STEP_INDEX_DISCHARGE_REST]
    
    if df_dchrest.empty:
        discharge_rest_delta_voltage_v.append(np.NaN)
    else:
        discharge_rest_delta_voltage_v.append(df_dchrest['voltage_v'].tail(1).item() - \
                                              df_dchrest['voltage_v'].head(1).item())
    
        
# Assign back to the DataFrame    
df_by_cyc['charge_cc_time_s'] = charge_cc_time_s
df_by_cyc['charge_cv_time_s'] = charge_cv_time_s
df_by_cyc['charge_rest_delta_voltage_v'] = charge_rest_delta_voltage_v
df_by_cyc['discharge_cc_time_s'] = discharge_cc_time_s
df_by_cyc['discharge_rest_delta_voltage_v'] = discharge_rest_delta_voltage_v

df_by_cyc

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,discharge_energy_wh,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh,charge_cc_time_s,charge_cv_time_s,charge_rest_delta_voltage_v,discharge_cc_time_s,discharge_rest_delta_voltage_v
cycle_index,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.000000,0.000000,0.192217,0.612502,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.081963,0.268955,0.199162,0.632655,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.163932,0.550408,0.206113,0.654522,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.245902,0.837872,0.213059,0.677289,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.327873,1.126976,0.220013,0.700393,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,1.257079,5.264991,1.254994,4.117342,2245.402472,9044.422953,2243.579642,7706.754823,401.580803,5073.072943,-0.007472,1807.048710,0.052956
1575,1.255640,5.259053,1.253413,4.112072,2246.658112,9049.682006,2244.833056,7710.866895,398.485173,5080.356886,-0.007635,1804.758365,0.053118
1576,1.254240,5.253376,1.252168,4.108112,2247.912352,9054.935383,2246.085223,7714.975007,396.081990,5079.580980,-0.007635,1802.986773,0.052956


In [106]:
plot_vars = ['discharge_capacity_ah', 
             'charge_cc_time_s', 
             'charge_cv_time_s', 
             'charge_rest_delta_voltage_v',
             'discharge_cc_time_s', 
             'discharge_rest_delta_voltage_v']

xvar = 'tot_discharge_capacity_ah'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 4), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(df_by_cyc[xvar], df_by_cyc[yvar])
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    if ax == axs[0]:
        ax.set_title(f'{device_name} \n {yvar}')
    else:
        ax.set_title(yvar)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process the RPT Data

In [107]:
# Assemble a list of cycle indices where the RPT starts

rpt_start_cycle_list = []

unique_cycles = df['cycle_index'].unique()

for cycle in unique_cycles:
    
    this_df = df[df['cycle_index'] == cycle]
    
    if this_df['step_index'].isin([RPT_STEP_INDEX_START]).any():
        
        rpt_start_cycle_list.append(cycle)
        
rpt_start_cycle_list

In [116]:
fh1, axs1 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh3, axs3 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh4, axs4 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)

cycle_index_list = []
c20_discharge_capacity_ah_list = []
c20_charge_capacity_ah_list = []
throughput_ah_list = []
dvdq_data_list = []
voltage_decay_at_top_list = []
voltage_decay_at_bot_list = []

for rpt_start_cycle in rpt_start_cycle_list:
    
    # Create a DataFrame to hold the current RPT data only
    this_rpt = df_rpt[(df_rpt['cycle_index'] >= rpt_start_cycle)]
    this_rpt = this_rpt[this_rpt['cycle_index'] <= rpt_start_cycle + 80]
    this_rpt['time_s'] = this_rpt['test_time_s'] - this_rpt['test_time_s'].iloc[0]
    
    # Extract the charge and discharge C/20 curves
    c20_charge_capacity_ah = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_CHARGE]['charge_capacity_ah']
    c20_charge_voltage_v   = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_CHARGE]['voltage_v']
    c20_discharge_capacity_ah = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_DISCHARGE]['discharge_capacity_ah']
    c20_discharge_voltage_v   = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_DISCHARGE]['voltage_v']
    
    voltage_decay_at_top = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_VOLTAGE_RELAX_TOP_12HR]['voltage_v']
    voltage_decay_at_bot = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_VOLTAGE_RELAX_BOT_2P5HR]['voltage_v']
    
    # Calculate the filtered dV/dQ outputs
    window_length = 101
    polyorder = 2
    Qf_d = signal.savgol_filter(c20_discharge_capacity_ah,window_length,polyorder)
    dQ_d = signal.savgol_filter(c20_discharge_capacity_ah,window_length,polyorder,1)
    Vf_d = signal.savgol_filter(c20_discharge_voltage_v,window_length,polyorder)
    dV_d = signal.savgol_filter(c20_discharge_voltage_v,window_length,polyorder,1)
    dVdQ_d = dV_d / dQ_d    
    dQdV_d = dQ_d / dV_d
    
    Qf_c = signal.savgol_filter(c20_charge_capacity_ah,window_length,polyorder)
    dQ_c = signal.savgol_filter(c20_charge_capacity_ah,window_length,polyorder,1)
    Vf_c = signal.savgol_filter(c20_charge_voltage_v,window_length,polyorder)
    dV_c = signal.savgol_filter(c20_charge_voltage_v,window_length,polyorder,1)
    dVdQ_c = dV_c / dQ_c    
    dQdV_c = dQ_c / dV_c
    
    this_dvdq_data = dict()
    this_dvdq_data['dch_q_ah'] = Qf_d
    this_dvdq_data['dch_v_ah'] = Vf_d
    this_dvdq_data['dch_dvdq'] = dVdQ_d
    this_dvdq_data['chg_q_ah'] = Qf_c
    this_dvdq_data['chg_v_ah'] = Vf_c
    this_dvdq_data['chg_dvdq_ah'] = dVdQ_c
        
    throughput_ah_list.append(df_agg['discharge_capacity_ah'].cumsum()[df_agg.index == rpt_start_cycle].values[0])
    dvdq_data_list.append(this_dvdq_data)
    c20_discharge_capacity_ah_list.append(c20_discharge_capacity_ah.max())
    c20_charge_capacity_ah_list.append(c20_charge_capacity_ah.max())
    cycle_index_list.append(rpt_start_cycle)
    voltage_decay_at_top_list.append(voltage_decay_at_top.tail(1).item() - voltage_decay_at_top.head(1).item())
    voltage_decay_at_bot_list.append(voltage_decay_at_bot.tail(1).item() - voltage_decay_at_bot.head(1).item())

    # Make the plots
    axs1[0].plot(this_rpt['time_s'], this_rpt['voltage_v'])
    axs1[1].plot(this_rpt['time_s'], this_rpt['step_index'])

    axs2[0].plot(c20_charge_capacity_ah, c20_charge_voltage_v)
    axs2[1].plot(c20_discharge_capacity_ah, c20_discharge_voltage_v)
    
    axs3[0].plot(Qf_c, dVdQ_c)
    axs3[1].plot(Qf_d, dVdQ_d)

    axs3[0].set_ylim([0, 0.75])
    axs3[1].set_ylim([-0.75, 0])
    
    axs4[0].plot(Vf_c, dQdV_c)
    axs4[1].plot(Vf_d, dQdV_d)
    
    
df_by_rpt = pd.DataFrame(list(zip(cycle_index_list, 
                                  throughput_ah_list, 
                                  c20_discharge_capacity_ah_list, 
                                  c20_charge_capacity_ah_list,
                                  voltage_decay_at_top_list,
                                  voltage_decay_at_bot_list,
                                  dvdq_data_list)), columns=['cycle_index', 
                                                             'tot_discharge_capacity_ah',
                                                             'c20_discharge_capacity_ah',
                                                             'c20_charge_capacity_ah',
                                                             'deltav_4p2v_12hr',
                                                             'deltav_3p0v_2p5hr',
                                                             'dvdq_data'])
    
    
df_by_rpt
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,cycle_index,tot_discharge_capacity_ah,c20_discharge_capacity_ah,c20_charge_capacity_ah,deltav_4p2v_12hr,deltav_3p0v_2p5hr,dvdq_data
0,1,0.192217,2.450101,2.462806,-0.009909,1.042874,"{'dch_q_ah': [-1.1103596010390055e-07, 0.00034..."
1,85,118.888660,2.384881,2.393803,-0.009097,0.960841,"{'dch_q_ah': [-7.3833006846892464e-06, 0.00034..."
2,166,231.831826,2.342456,2.349416,-0.008934,0.972212,"{'dch_q_ah': [2.516420756191242e-06, 0.0003496..."
3,297,451.593750,2.286034,2.289952,-0.008934,0.923480,"{'dch_q_ah': [4.3795004310368883e-08, 0.000347..."
4,426,664.142368,2.236874,2.236530,-0.010071,0.902850,"{'dch_q_ah': [1.0170952482848761e-06, 0.000348..."
5,554,870.826377,2.197956,2.184731,-0.010396,0.910972,"{'dch_q_ah': [-1.0398587716076748e-06, 0.00034..."
6,682,1071.104738,2.147890,2.145963,-0.011371,0.888880,"{'dch_q_ah': [2.623398695078341e-05, 0.0003710..."
7,809,1264.972380,2.102234,2.092820,-0.011696,0.860940,"{'dch_q_ah': [4.917601529560142e-07, 0.0003476..."
8,935,1450.507178,2.072768,2.063589,-0.012995,0.834787,"{'dch_q_ah': [3.2212947392760663e-06, 0.000350..."
9,1060,1627.787950,2.042194,2.024239,-0.013320,0.804897,"{'dch_q_ah': [-8.012415311491854e-07, 0.000346..."


,cycle_index,tot_discharge_capacity_ah,c20_discharge_capacity_ah,c20_charge_capacity_ah,dvdq_data
0,1,0.192217,2.450101,2.462806,"{'dch_q_ah': [-1.1103596010390055e-07, 0.00034..."
1,85,118.888660,2.384881,2.393803,"{'dch_q_ah': [-7.3833006846892464e-06, 0.00034..."
2,166,231.831826,2.342456,2.349416,"{'dch_q_ah': [2.516420756191242e-06, 0.0003496..."
3,297,451.593750,2.286034,2.289952,"{'dch_q_ah': [4.3795004310368883e-08, 0.000347..."
4,426,664.142368,2.236874,2.236530,"{'dch_q_ah': [1.0170952482848761e-06, 0.000348..."
5,554,870.826377,2.197956,2.184731,"{'dch_q_ah': [-1.0398587716076748e-06, 0.00034..."
6,682,1071.104738,2.147890,2.145963,"{'dch_q_ah': [2.623398695078341e-05, 0.0003710..."
7,809,1264.972380,2.102234,2.092820,"{'dch_q_ah': [4.917601529560142e-07, 0.0003476..."
8,935,1450.507178,2.072768,2.063589,"{'dch_q_ah': [3.2212947392760663e-06, 0.000350..."
9,1060,1627.787950,2.042194,2.024239,"{'dch_q_ah': [-8.012415311491854e-07, 0.000346..."


In [118]:
plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['c20_discharge_capacity_ah'], marker='o', label='C/20 DCH')
plt.plot(df_by_cyc['tot_discharge_capacity_ah']/2.5, df_by_cyc['discharge_capacity_ah'], label='1C DCH')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name)

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_3p0v_2p5hr'], marker='o', label='$\Delta$V, 3.0V, 2.5hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name)

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_4p2v_12hr'], marker='o', label='$\Delta$V, 4.2V, 12hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'UMBL2022FEB_CELL152028')

In [111]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 3), sharex=True)


for yvar, ax in zip(plot_vars, axs):
    ax.plot(df_rpt[xvar], df_rpt[yvar], ls='', marker='o', ms=0.3, c='r')
    ax.plot(df_cyc[xvar], df_cyc[yvar], ls='', marker='o', ms=0.3, c='b')
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);
axs[0].set_ylim((2.7, 4.22))
axs[0].set_title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …